In [9]:
from IPython.display import display, HTML
display(HTML("<style>pre{white-space: pre !important;}</style>"))

In [10]:
import pandas as pd
import findspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [11]:
findspark.init()

In [16]:
spark = SparkSession.builder \
    .appName("Fire Incident") \
    .master("local[1]") \
    .config("spark.driver.host", "host.docker.internal") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "120s") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.executor.instances", "1") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "512m") \
    .config("spark.driver.memory", "512m") \
    .getOrCreate()

In [18]:
df = spark.read.csv('../data/raw/Fire_Incidents_20240516.csv', header=True)
df.show()

+---------------+---------------+---------+--------------------+-------------+-----------+--------------------+--------------------+--------------------+-------------+-------+---------+------------+----+-----------------+---------------------+---------+-------------+-----------+---------------+-------------------+-----------------------+-----------------------+---------------+-------------+-------------------+-----------------+----------------+--------------------+----------+--------------------+----------------------+------------------+--------------------------+--------------------+-------------------+--------------+-----------------------+-------------------------+-----------+------------------+--------------------------------------+--------------+----------------+--------------------+-----------+---------------+------------------------------------+----------------------------------------+----------------------------------+------------------------------------+-----------------+-----

In [30]:
spark.stop()